In [1]:
%matplotlib inline

# -*- coding: utf-8 -*-

import matplotlib
import matplotlib.pyplot as plt
import os
import pandas as pd
from redminelib import Redmine

environ_value = dict(os.environ)

# Validate the REDMINE_* before
redmine_url = environ_value['REDMINE_URL']
redmine_username = environ_value['REDMINE_USERNAME']
redmine_password = environ_value['REDMINE_PASSWORD']
project_name = 'biz_req'
redmine = Redmine(redmine_url, username=redmine_username,
                  password=redmine_password)
project = redmine.project.get(project_name)
data = []

for i in project.issues:
    data.append([
#         i.attachments,
         i.author.name,
#         i.changesets,
#         i.children,
         i.created_on.isoformat(),
#         i.custom_fields,
         i.description,
#         i.done_ratio,
         i.id,
#         i.journals,
         i.priority.name,
         i.project.name,
#         i.relations,
#         i.start_date,
         i.status.name,
         i.subject,
#         i.time_entries,
         i.tracker.name,
         i.updated_on.isoformat(),
#         i.watchers
     ])

dp = pd.DataFrame(data,
                  columns=['author', 'created time', 'description', 'id',
                           'priority', 'project', 'status', 'subject',
                           'tracker', 'updated time'])
dp.head()

,author,created time,description,id,priority,project,status,subject,tracker,updated time
0,陈 巧敏,2017-08-24T09:10:48,,15112,普通,九州云Scrum PBI,进行中,[5.1.0] 普通用户处于申请云硬盘状态，恢复备份时不应该有创建新云硬盘的选项,错误,2017-08-24T09:11:32
1,马 磊,2017-08-24T09:03:19,,15110,普通,九州云Scrum PBI,进行中,5.2.0 发布产品打包RC2和部署,部署环境,2017-08-24T09:07:31
2,马 磊,2017-08-24T09:02:33,,15108,普通,九州云Scrum PBI,已解决,5.2.0 发布产品打包RC1和部署,部署环境,2017-08-24T09:07:29
3,马 磊,2017-08-24T09:01:33,,15106,普通,九州云Scrum PBI,新建,5.2.0 产品打包和部署,PBI,2017-08-24T09:07:25
4,陈 耿昆,2017-08-24T08:13:49,,15104,普通,九州云Scrum PBI,进行中,[5.1.0] 云主机监控时间轴比实际时间晚8小时,错误,2017-08-24T09:46:02


In [ ]:
dp.groupby(u'status').size().plot()